<a href="https://colab.research.google.com/github/fender8185/Stock-predict/blob/main/lr2stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install FinMind==1.5.4

In [25]:
import tensorflow as tf
import numpy as np
import FinMind
from FinMind.data import DataLoader
from FinMind import plotting
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [26]:
stock_id = '2330'
start_date = '2018-01-01'
end_date = '2023-08-10'
data_loader = DataLoader()
stock_data = data_loader.taiwan_stock_daily(stock_id, start_date, end_date)
stock_data = data_loader.feature.add_kline_institutional_investors(
    stock_data
)
stock_data = data_loader.feature.add_kline_margin_purchase_short_sale(
    stock_data
)
all_dates = stock_data['date'].values[1:]
selected_columns = ['Foreign_Investor_diff','Margin_Purchase_diff','Investment_Trust_diff', 'close' ]
filtered_data = stock_data[selected_columns]
numpy_array = filtered_data.values
#原始的股價最後畫圖用的
start_price = X_test[0, -1]


In [27]:
# 計算價差
price_diff = numpy_array[1:, -1] - numpy_array[:-1, -1]
# 更新 y
y = price_diff
# X 仍然是前一天的特徵值，但需要去掉最後一天的數據，以確保與 y 的長度相同
X = numpy_array[:-2,:-1]

split_index = int(len(X) * 0.8)  # 80% 資料作為訓練集
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = all_dates[:split_index]
test_dates = all_dates[split_index+1:]  # 因為 y 是從第二天開始的，所以 test_dates 也需要從第二天開始

#normalization 效果竟然比standard效果好？
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# 縮放目標值
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()
'''
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()
'''
# 定義模型參數
W = tf.Variable(tf.random.normal([X_train_scaled.shape[1], 1], dtype=tf.float64), dtype=tf.float64, name="weight")
#b = tf.Variable(tf.random.normal([1], dtype=tf.float64), dtype=tf.float64, name="bias")
b = tf.Variable(tf.zeros([1], dtype=tf.float64), dtype=tf.float64, name="bias")

# 定義模型
def linear_regression(X):
    return tf.matmul(X, W) + b

# 定義損失
def mean_square(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - tf.cast(y_true, tf.float64)))
def mean_absolute_error(y_pred, y_true):
    return tf.reduce_mean(tf.abs(y_pred - tf.cast(y_true, tf.float64)))

# 定義優化器
#optimizer = tf.optimizers.SGD(learning_rate=0.1)
optimizer = tf.optimizers.Adam(learning_rate=0.0001)

# 訓練模型
n_epochs = 14000
for epoch in range(n_epochs):
    with tf.GradientTape() as tape:
        y_pred = linear_regression(X_train_scaled)
        loss = mean_absolute_error(y_pred, y_train_scaled)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
# 評估模型
y_pred_test_scaled = linear_regression(X_test_scaled)
#print(X_test_scaled)
#print(y_pred_test_scaled)
#mse_test = mean_absolute_error(y_pred_test_scaled, y_test_scaled)
#print(f"Mean Squared Error on Scaled Test Data: {mse_test.numpy()}")
print(y_pred)
# 反縮放預測值回原始尺度
y_pred_test_original = scaler_y.inverse_transform(y_pred_test_scaled.numpy())
y_pred_train_original = scaler_y.inverse_transform(y_pred.numpy())
#mse_test_original = mean_squared_error(y_test, y_pred_test_original)
#print(f"Mean Squared Error on Original Test Data: {mse_test_original}")

#覺得怪怪的但還是做了
# 使用模型對縮放後的訓練數據進行預測
#y_pred_train_scaled = linear_regression(X_train_scaled)
#y_pred_train_original = scaler_y.inverse_transform(y_pred_train_scaled.numpy())

#print(y_pred_train_original)

tf.Tensor(
[[0.4663156 ]
 [0.49574925]
 [0.42998203]
 ...
 [0.44932008]
 [0.44063275]
 [0.45498113]], shape=(1090, 1), dtype=float64)


In [30]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

# 添加訓練數據到第一個子圖
fig.add_trace(go.Scatter(x=train_dates, y=y_train, mode='lines', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=train_dates, y=y_pred_train_original.flatten(), mode='lines', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

# 添加測試數據的真實值到第二個子圖
fig.add_trace(go.Scatter(x=test_dates, y=y_test, mode='lines', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=test_dates, y=y_pred_test_original.flatten(), mode='lines', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)

fig.update_layout(hovermode='x unified')

# 顯示圖形
fig.show()


In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 使用原始的股價計算累積的股價
cumulative_y_train_original = numpy_array[:split_index, -1]
cumulative_y_test_original = numpy_array[split_index+1:-1, -1]

# 昨天的實際close＋predict
cumulative_y_pred_train = cumulative_y_train_original[:-1] + y_pred_train_original[1:].flatten()
cumulative_y_pred_test = cumulative_y_test_original[:-1] + y_pred_test_original[1:].flatten()

fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

# 添加訓練數據到第一個子圖
fig.add_trace(go.Scatter(x=train_dates, y=cumulative_y_train_original, mode='lines', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=train_dates[1:], y=cumulative_y_pred_train, mode='lines', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

# 添加測試數據的真實值到第二個子圖
fig.add_trace(go.Scatter(x=test_dates, y=cumulative_y_test_original, mode='lines', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=test_dates[1:], y=cumulative_y_pred_test, mode='lines', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)


# 設置懸停模式
fig.update_layout(hovermode='x unified')
# 顯示圖形
fig.show()
